In [23]:
# https://stackoverflow.com/questions/34837707/how-to-extract-text-from-a-pdf-file

import fitz # install using: pip install PyMuPDF
from os import listdir
import re

In [4]:
def get_text(pdfpath: str) -> str:
    with fitz.open(pdfpath) as doc:
        text = ""
        for page in doc:
            text += page.get_text()
    return text

In [15]:
# Get pdf paths
path = '../data/2023_국가공무원_9급_공채/'
file_list = [path+t for t in listdir(path)]
file_list[:3]

['../data/2023_국가공무원_9급_공채/230408 국가 9급 건축계획-나.pdf',
 '../data/2023_국가공무원_9급_공채/230408 국가 9급 건축구조-나.pdf',
 '../data/2023_국가공무원_9급_공채/230408 국가 9급 경제학개론-나.pdf']

In [22]:
# Get texts from pdf paths
text_list = list( map(get_text, file_list) )
text_list[:2]

['2023년도 국가공무원 9급 공채 필기시험\n건축계획\n나 책형\n1쪽\n건축계획\n1. 건축가와 주요 사상 및 대표 작품의 연결이 옳지 않은 것은?\n① 프랭크 로이드 라이트(Frank Lloyd Wright) － 유기적 건축 －\n낙수장(Falling Water)\n② 르 꼬르뷔제(Le Corbusier) － 근대건축의 5원칙 － 라투레트 \n수도원(Sainte Marie de La Tourette)\n③ 미스 반 데어 로에(Mies van der Rohe) － 적을수록 풍부하다\n(Less is more) － 시그램 빌딩(Seagram Building)\n④ 필립 존슨(Philip Johnson) －지역주의－로이드 보험 본사(Lloyd’s \nof London)\n2. 기후대에 따른 토속건축에 대한 설명으로 옳은 것은?\n① 고온건조기후에서는 일사가 충분하므로 이를 최대한 활용하기 \n위해 개구부의 수가 많고 크기 또한 크다.\n② 고온다습기후에서는 증발에 의한 냉각효과가 잘 일어나므로 \n습공기의 실내 체류 시간이 최대한 길게 설계되었다.\n③ 온난기후에서는 따뜻한 기후가 유지되므로 처마 등의 차양으로 \n연중 최대한 일사가 들지 않도록 하였다.\n④ 한랭기후에서는 열손실을 최소로 하는 것이 중요하므로 용적에 \n대한 표면적의 비율이 최소화되었다.\n3. 학교건축계획에서 교과교실형에 대한 설명으로 옳은 것은?\n① 각 학급이 전용 일반교실을 가지며 특정 교과는 특별교실을 두고 \n운영한다.\n② 각 교과의 순수율이 높은 교실이 주어지며 시설의 수준이 높아진다.\n③ 학생의 이동이 적으며 교실 이용률이 100 %라 하더라도 반드시 \n순수율이 높다고 할 수 없다.\n④ 초등학교 저학년에 가장 적합하며 안정적인 생활을 위한 홈베이스가 \n필요하다.\n4. 난방방식에 대한 설명으로 옳지 않은 것은?\n① 온수난방은 난방 휴지기간이 길면 동결의 우려가 있으나 증기난방에 \n비하여 쾌감도는 높다.\n② 증기난방은 현열을 이용하므로 배관 관경이 크고 열의 운반

In [169]:
def get_question_type(problem:str) -> str:
    if ('설명으로' in problem):     return 'describe'
    if ('모두 고르면' in problem):  return 'choose_all'   
    if ('고르면' in problem):       return 'choose'
    if ('나열' in problem):         return 'list'          # 모델 학습에 적합해보이지 않음.
    if ('연결' in problem):         return 'connect'       # 모델 학습에 적합해보이지 않음.
    if ('빈칸' in problem) \
    or ('(가)' in problem):         return 'blank'         # 모델 학습에 적합해보이지 않음.
    
    return 'none'


def get_correct_wrong(problem:str) -> int:
    if ('옳은' in problem) \
    or ('바르게' in problem): 
        return 1
    
    if ('옳지 않은' in problem) \
    or ('않는' in problem):
        return -1
    
    return  0

In [171]:
cleaned_text_list = []

for text in text_list:
    where = text.split('\n')[:2]
    
    text = text.replace('\n', ' ')
    
    # Find question
    problems = re.findall('\d+\.[\s@+].*?\s*?①', text)
    problems = [i[:-2] for i in problems]   # remove ' \n ①'
    
    
    problems.append(' ')                    # add blank for preventing index error
    
    for i in range(len(problems) -1 ):      # -1 b/c .append(' ')
        problem = text[ text.rfind(problems[i]):text.rfind(problems[i+1]) ]
        
        cleaned_text_list.append({
            'where': where[0],
            'subject': where[1],
            'question': problems[i],
            'choices': problem.replace(problems[i], ''),
            'question_type': get_question_type(problem),
            'correct_wrong': get_correct_wrong(problem),
        })
    break

cleaned_text_list

[{'where': '2023년도 국가공무원 9급 공채 필기시험',
  'subject': '건축계획',
  'question': '1. 건축가와 주요 사상 및 대표 작품의 연결이 옳지 않은 것은?',
  'choices': ' ① 프랭크 로이드 라이트(Frank Lloyd Wright) － 유기적 건축 － 낙수장(Falling Water) ② 르 꼬르뷔제(Le Corbusier) － 근대건축의 5원칙 － 라투레트  수도원(Sainte Marie de La Tourette) ③ 미스 반 데어 로에(Mies van der Rohe) － 적을수록 풍부하다 (Less is more) － 시그램 빌딩(Seagram Building) ④ 필립 존슨(Philip Johnson) －지역주의－로이드 보험 본사(Lloyd’s  of London) ',
  'question_type': 'connect',
  'correct_wrong': -1},
 {'where': '2023년도 국가공무원 9급 공채 필기시험',
  'subject': '건축계획',
  'question': '2. 기후대에 따른 토속건축에 대한 설명으로 옳은 것은?',
  'choices': ' ① 고온건조기후에서는 일사가 충분하므로 이를 최대한 활용하기  위해 개구부의 수가 많고 크기 또한 크다. ② 고온다습기후에서는 증발에 의한 냉각효과가 잘 일어나므로  습공기의 실내 체류 시간이 최대한 길게 설계되었다. ③ 온난기후에서는 따뜻한 기후가 유지되므로 처마 등의 차양으로  연중 최대한 일사가 들지 않도록 하였다. ④ 한랭기후에서는 열손실을 최소로 하는 것이 중요하므로 용적에  대한 표면적의 비율이 최소화되었다. ',
  'question_type': 'describe',
  'correct_wrong': 1},
 {'where': '2023년도 국가공무원 9급 공채 필기시험',
  'subject': '건축계획',
  'question': '3. 학교건축계획에서 교과교실형에 대한 설명으로 옳은 것은?',
  